In [1]:
import os
import json
# from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import autogen
# from autogen import AssistantAgent, Agent, UserProxyAgent, ConversableAgent

# from autogen.agentchat.contrib.img_utils import get_image_data, _to_pil
# from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent

# from termcolor import colored
import random

from autogen.code_utils import DEFAULT_MODEL, UNKNOWN, content_str, execute_code, extract_code, infer_lang

#
import os
from pathlib import Path
# import matplotlib.pyplot as plt
os.chdir("../../")
Path.cwd()

/home/mymm_psu_gmail_com/miniconda3/envs/dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PosixPath('/home/mymm_psu_gmail_com/hackathon/rag-agents/multimodal-conv-agents')

In [ ]:
from omegaconf import OmegaConf
from hydra import compose, initialize
from hydra.core.global_hydra import GlobalHydra

from agents.ontology.config.dialogue import ConversationConfig, PodcastConfig # 

# Clear Hydra's global state if it is already initialized
if GlobalHydra.instance().is_initialized():
  GlobalHydra.instance().clear()
with initialize(config_path="../../conf/dialogue"):
  config = compose(config_name="default")
  # Convert the OmegaConf config to the Pydantic model
  cfg: ConversationConfig = ConversationConfig(
    **OmegaConf.to_container(config, resolve=True)
  )

# Print the configuration to verify
print(cfg)
%load_ext autoreload
%autoreload 2

config_list_gemini = autogen.config_list_from_json(
    "conf/OAI_CONFIG_LIST.txt",
    filter_dict={
        "model": ["gemini-1.5-pro"],
    },
)

# ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"]
# ["gemini-pro-vision"]
llm_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_gemini,
    "timeout": 120,
}

In [3]:
from langchain.output_parsers import PydanticOutputParser

from agents.ontology.config.dialogue import *
from agents.ontology.parser.dialogue import monologue_parser, podcast_parser

In [4]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()
initializer = autogen.UserProxyAgent(
    name="init", 
    code_execution_config=False,
)

coder = autogen.AssistantAgent(
    name="retrieve_coder",
    llm_config=llm_config,
    system_message="""You are the Coder. 
    You write python/shell code to solve the task presented. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
    Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
    If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try. Ensure proper error handling such that an approrpriate format of  results is returned with the error code.
    """,
)
research_coder = autogen.AssistantAgent(
    name="research_coder",
    llm_config=llm_config,
    human_input_mode="NEVER",
    system_message="""You are the Coder. You write python/shell code to gather relevant web information for the task. Provide the code in a code block that is intended to be executed by the executor.
    The following are the guidelines: 
    The user can't modify your code. So do not suggest incomplete code which requires others to modify. 
    Don't include multiple code blocks in one response. 
    Do not ask others to copy and paste the result. Check the execution result returned by the executor. If the result indicates there is an error, fix the error and output the code again. 
    Suggest the full code instead of partial code or code changes. 
    If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try. 
    Ensure proper error handling such that an appropriate format of results is returned with the error code. 
    Do not use any method that requires an external API key to work.
    """,
)
executor = autogen.UserProxyAgent(
    name="executor",
    system_message="Executor. Execute the code written by the Coder and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "outputs/code",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)
informer = autogen.AssistantAgent(
    name="informer",
    llm_config=llm_config,
    human_input_mode="NEVER",
    system_message="""Provide the summarized biograpy of the guests in the conversation to the podcast hosts for starting the conversation. The summary must include their most known achievements, personality and relevant news as context that most informs the character of the guests in the conversation.""",
)

script_parser = autogen.AssistantAgent(
    name="json_parser",
    llm_config=llm_config,
    human_input_mode="NEVER",
    system_message=f"Ensure all '```json' is converted into a valid JSON. {podcast_parser.get_format_instructions()}",
)

In [7]:
podcast_config = cfg.podcast_config

podcast_hosts = [
    autogen.ConversableAgent(
        name=host.name,
        is_termination_msg=termination_msg,
        human_input_mode="NEVER",
        code_execution_config=False,  # we don't want to execute code in this case.
        llm_config=cfg.llm_config.model_dump(),
        description=host.description,
        system_message=f"""As yourself: {host.name}, respond to the conversation.
        {monologue_parser.get_format_instructions()}""",
    )
    for host in podcast_config.character_cfg.hosts
]
podcast_gents = [
    autogen.ConversableAgent(
        name=guest.name,
        llm_config=cfg.llm_config.model_dump(),
        human_input_mode="NEVER",
        system_message=f"""As yourself: {guest.name}, respond to the conversation. {monologue_parser.get_format_instructions()}""",
        description=guest.description,
    ) for guest in podcast_config.character_cfg.guests
]

In [8]:
from agents.dialogue.transition import get_state_transition
MAX_ROUND=10

groupchat = autogen.GroupChat(
    agents = [ initializer, research_coder, executor, informer ] + podcast_hosts + podcast_gents + [ script_parser ],
    messages = [],
    max_round=MAX_ROUND ,
    speaker_selection_method=get_state_transition(
        podcast_config, "podcast.default"
    ),
    # speaker_selection_method="round_robin"
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [10]:
characters_str = ", ".join(podcast_config.character_cfg.guest_names)
topic = "Democracy"
chat_result = initializer.initiate_chat(
    manager, 
    message=f"You are going to prepare the host for a podcast among: {characters_str} in a real-life conversation about {topic}."
)

init (to chat_manager):

You are going to prepare the host for a podcast among: Harry Potter, Iron Man, Darth Vader, Alan Turing, Albert Einstein, Genghis Khan in a real-life conversation about Democracy.

--------------------------------------------------------------------------------

Next speaker: research_coder

research_coder (to chat_manager):

```python
import requests
from bs4 import BeautifulSoup

def get_guest_info(guest_name):
    """Fetches a brief description of a person from Wikipedia.

    Args:
        guest_name: The name of the person to search for.

    Returns:
        A string containing a brief description of the person, or
        an error message if an error occurs.
    """
    try:
        url = f"https://en.wikipedia.org/wiki/{guest_name}"
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')
        paragraphs = soup.find_all('p')

        # Get the first non-empty paragraph
   

/home/mymm_psu_gmail_com/hackathon/rag-agents/multimodal-conv-agents/agents/utils/registry.py:13: UserWarning: Class name 'podcast.default' already registered. Overwriting existing entry.
  warnings.warn(f"Class name '{name}' already registered. Overwriting existing entry.", UserWarning)


executor (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 
Welcome to the podcast! Today's topic is Democracy, and we have an interesting mix of guests:
- **Harry Potter:** Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. The novels chronicle the lives of a young wizard, Harry Potter, and his friends, Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry. The main story arc concerns Harry's conflict with Lord Voldemort, a dark wizard who intends to become immortal, overthrow the wizard governing body known as the Ministry of Magic, and subjugate all wizards and Muggles (non-magical people).
- **Iron Man:** Iron Man is a superhero appearing in American comic books published by Marvel Comics. Co-created by writer and editor Stan Lee, developed by scripter Larry Lieber, and designed by artists Don Heck and Jack Kirby, the character first appeared in Tales of Suspense #39 in 1962 a

In [ ]:
json_data = chat_result.chat_history[-1].get('content').replace("```json", "").replace("```", "")
script_json = json.loads(json_data)

In [ ]:
from datetime import datetime
import json

def save_conversation(conv_json, output_dir=Path("outputs/conversations")):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    # Get the current datetime string
    current_datetime_str = datetime.now().strftime('%Y%m%d_%H%M%S')
    # Create the output file path
    output_file_path = os.path.join(output_dir, f'script_json_{current_datetime_str}.json')
    # Write the list of nested JSON objects to the file
    with open(output_file_path, 'w') as output_file:
        json.dump(conv_json, output_file, indent=4, ensure_ascii=False)
    print(f"JSON list saved to {output_file_path}")

In [ ]:
save_conversation(script_json)

JSON list saved to outputs/conversations/script_json_20240715_214605.json
